# Road Traffic Accidents Analysis UK (2019)

Investigative analysis and predictive modelling based on road traffic accidents reported in the United Kingdom in 2019.

The data consists of three individual but related datasets

- **Accident dataset**  Accidents: 32 variables, detailing location, time, date, lighting, weather, road conditions and other variables. The unique accident index identifies each observation and make up one of 117,536 collisions
- **Vehicle dataset** Contains details of vehicles involved in the accidents.
- **Casualties dataset** 16 columns with information about casualties involved in accidents.

In [ ]:
#Importing the required libraries
import numpy as np #for linear algebra/data preprocessing
import pandas as pd #for data preprocessing
import matplotlib.pyplot as plt #For visualization
plt.style.use("fivethirtyeight")#For styling the plots
import seaborn as sns #for visualization
import datetime
import warnings
warnings.filterwarnings('ignore')
import math

In [ ]:
from sklearn.cluster import KMeans #For clustering
from scipy.stats import mannwhitneyu, shapiro, ttest_ind #For statistical tests
from statsmodels.stats import weightstats as stests #For statistical tests
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler #For Scaling
from sklearn.preprocessing import LabelEncoder, OneHotEncoder #For Encoding
from scipy.spatial.distance import euclidean, cityblock #For distance measurement
import category_encoders as ce #For Encoding
from sklearn.decomposition import PCA #For Principal Components Analysis
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression #To check for feature importance

In [ ]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedStratifiedKFold, RepeatedKFold #To split data
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,GradientBoostingClassifier,GradientBoostingRegressor# Algorithms for predictions
from xgboost import XGBClassifier 
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor # predicting algorithms
from sklearn.linear_model import LinearRegression, LogisticRegression # predicting algorithms
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, f1_score, classification_report, confusion_matrix #For evaluating built models
from sklearn.ensemble import StackingClassifier, StackingRegressor
from sklearn.model_selection import cross_val_score
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
# Load datasets
Accidents = pd.read_csv('Road Safety Data - Accidents 2019.csv', parse_dates = True)# Read in the Accidents dataset
casualties = pd.read_csv('Road Safety Data - Casualties 2019.csv', parse_dates = True)# Read in the casualties dataset
vehicles = pd.read_csv('Road Safety Data- Vehicles 2019.csv', parse_dates = True)# Read in the vehicles dataset

## Initial EDA
Having imported appropiate libraries such as Pandas, Numpy, Matplotlib, Seaborn and Scipy, The dataset is imported to and a profile report is generated to identify inconsistences

In [ ]:
#from pandas_profiling import ProfileReport
#profile = ProfileReport(accidents, title="Accidents Profile Report")
#profile.to_notebook_iframe()

## Data Cleaning
***
Having imported the appropiate libraries such as Pandas, Numpy, Matplotlib, Seaborn and Scipy, I imported the dataset to get an idea of what I'm working with
Associated Steps:

- Examine data information
- Inspect data for missing values.
- Handling Missing values.
- Merge datasets, re-examine data information and handle outstanding issues

In [ ]:
Accidents.info()

In [ ]:
vehicles.info()

In [ ]:
casualties.info()

From the datasets we can identify one common feature; Accident_Index. 

In [ ]:
def Check_null(data):
    '''
    This function calculates the number of missing values in columns of a dataset. 
    It takes a dataframe as its only parameter. 
    It is implemented using the pandas object method, isnull(). The method method returns a DataFrame object
    where all the values are replaced with a Boolean value True for NULL values, and otherwise False.
    The rest of the function sums up the count of null values and appends them to variable (miss) which the function returns'''
    
    miss = data.isnull().sum() # Compute the percentage and assign it to a variable
    return miss # Return variable

In [ ]:
#Check volume of missing values per column in accidents.
Check_null(Accidents)


In [ ]:
#Check volume missing values per column in Vehicles.
Check_null(vehicles)

In [ ]:
#Check volume missing values per column in casualties.
Check_null(casualties)

The Accident dataset is the only dataset with missing values in various columns. The columns with missing values are location and time features. 

#### Handling
As the percentages of missing values of said features is less than one percent and the information cannot be inferred from the rest of the dataset, applying forward fill or backward fill will not negatively affect or skew the data. The methods will replace the missing cells with duplicates from the existing data.

Forward fill and Backward fill fetch the value from the index before or after the missing value index respectively to fill.

In this case, the scale of the change is small, the methods therefore ensure the shape of the dataframe will remain unaffected. The methods are also ideal for temporal features like time

Since both affected features need not be normally distributed and the shape of distribution will be retained, forward and backward filling is adequate for the problem.

In [ ]:
def fill_missing(data, use_method='ffill', way='dataframe', column=None):
    '''
        Function fills missing values in a dataset or individual column, witha scalar value using multiple methods. 
        It accepts the following parameters; 
        a dataframe, 
        a method to fill the missing values- set to ffill by default, 
        way - data structure - dataframe or dataframe column.
        column- column name - set to none by default.

        It applies pandas fillna methods; ffill = forward fill,  backfill == backward fill 
        and other statistically significant values including;
        The median score
        The mean score
        The modal value 
    '''
    if way == 'dataframe': # Condition to check way as dataframe
        if use_method == 'ffill': # Condition to check for use method as forward fill
            data.fillna(method='ffill', inplace = True) # forward fill the missing value
            return data.isnull().sum() # Return the sum of missing values
        elif use_method == 'backfill': # Condition to check for use method as backward fill
            data.fillna(method='backfill', inplace = True) # backward fill the missing value
            return data.isnull().sum() # Return the sum of missing values
        else :
            data.fillna(use_method, inplace = True) # use impute to fill the missing value
            return data.isnull().sum() # Return the sum of missing values
    elif way == 'feature': # Condition to check way as feature
        if use_method == 'ffill': # Condition to check for use method as forward fill
            data[column].fillna(method='ffill', inplace = True) # forward fill the missing value
            return data[column].isnull().sum() # Return the sum of missing values
        elif use_method == 'backfill': # Condition to check for use method as backward fill
            data[column].fillna(method='backfill', inplace = True) # backward fill the missing value
            return data[column].isnull().sum() # Return the sum of missing values
        elif use_method == 'median': # Condition to check for use method as median
            data[column].fillna(data[column].median(), inplace = True) # Use median to fill the missing value
            return data[column].isnull().sum() # Return the sum of missing values
        elif use_method == 'mean': # Condition to check for use method as mean
            data[column].fillna(data[column].mean(), inplace = True) # Use mean to fill the missing value
            return data[column].isnull().sum() # Return the sum of missing values
        elif use_method == 'mode': # Condition to check for use method as mode
            data[column].fillna(data[column].mode()[0], inplace = True) # Use mode to fill the missing value
            return data[column].isnull().sum() # Return the sum of missing values
        else :
            data[column].fillna(use_method, inplace = True) # use impute to fill the missing value
            return data[column].isnull().sum() # Return the sum of missing values
        

In [ ]:
#Filling Missing Values
fill_missing(Accidents)
fill_missing(Accidents, 'backfill')

### Merging the datasets.

In order to perform the required analysis for relevant insights to some questions in this project, there is a need to compare common features within the different data frames. The datasets are therefore merged to achieve to facilitate this comparison.

applying the Pandas .merge() method requires a common index. The **Accident Index** column is sufficient for our three datasets. Resulting missing values from the merge will be handled with the previously defined method to avoid loss of data

In [ ]:
#Check Unique values for the common feature
print(Accidents['Accident_Index'].nunique())
print(vehicles['Accident_Index'].nunique())
print(casualties['Accident_Index'].nunique())

The result of the above cell gives the inference that the three datasets have very similar classes of their common feature, which further justifies joining on that feature

In [ ]:
# Merging datasets
casualty_vehicle = pd.merge(vehicles, casualties, on = "Accident_Index", how = "outer") #To merge Casualties and Vehicles datasets together

In [ ]:
df = pd.merge(Accidents,casualty_vehicle, on = "Accident_Index", how = "left") #To merge casualty_vehicles and accidents together

In [ ]:
# drop duplicated rows
df.drop_duplicates(keep='first',inplace=True)

In [ ]:
df.info()

In [ ]:
df['Accident_Index'].nunique()

In [ ]:
Check_null(df)

In handling the missing values, we apply forward fill and backward fill since the datasets a common feature; **Accident_Index**. This means that their index will be determined by Accident_Index and all features will match across columns

In [ ]:
fill_missing(df) #For forward fill
fill_missing(df, 'backfill') #For backward fill

In [ ]:
Check_null(df) #Check for missing values

# Exploratory Data Analysis
***
As the data is now relatively clean to a reasonable extent, we explore the data for useful insights to our project problems. 

In [ ]:
#make a copy to manipulate for analysis
Analysis = df.copy() #Create a copy of the merged dataset to use for some analysis
Analysis.info()

To effectively analyze the time related features, the values of Time column are converted to datetime objects with the Datetime Python module.

NB. Accidents only related analysis wil be carried out on accidents dataset only to ensure unique accidents cases are examined

In [ ]:
# Define datetime objects

# parse the date values to real date objects
Accidents['Date'] = pd.to_datetime(Accidents['Date'])

Accidents['Real_time'] = pd.DatetimeIndex(Accidents['Time']) #Convert to datetime object

# convert to decimal time
def convert_time (time):
    dec_time = time.hour + time.minute/60
    return dec_time

Accidents['decimal_time'] = Accidents['Real_time'].apply(lambda x: convert_time(x))

# month
Accidents['Month'] = Accidents['Date'].dt.month

# week
Accidents['week'] = Accidents['Date'].dt.week

#convert to exact time of week.
Accidents['decimal_week'] = Accidents['Date'].dt.week + Accidents['Date'].dt.day/7
 
# hour
Accidents['Hour'] = Accidents['Real_time'].dt.hour


In [ ]:
# Preview engineered date features
Accidents[['Date', 'Month', 'week', 'Day_of_Week', 'Hour', 'decimal_time']].head(10)

## A. Significant periods of all road traffic accidents.

According to [UK driving skills][]  occur more often during rush hour than other times of day and in a news report by [BBC][] rush hour was defined to be between 06:00 and 09:00 and 16:00 to 19:00 on weekdays.

### Are there significant hours of the day, and days of the week, on which accidents occur?

>Since our entire data represents the population of the UK and not just a sample, we can make our statiscal inference by directly reviewing the rate of accidents within our periods of interest [Statistical Inference][] We also examine the descriptive statistics of accidents relative to the period

The Hour feature has 24 unique values equivalent to the 24 hours in a day.

The Day_of_Week feature has 7 unique values equivalent to the 7 days in a week.

[UK driving skills]: https://www.ukdrivingskills.co.uk/avoiding-accidents-in-rush-hour/
[BBC]: https://www.bbc.co.uk/news/uk-england-42917201
[Statistical Inference]: http://vargas-solar.com/data-centric-smart-everything/statistical-inference/. 

In [ ]:
def rate(param1, param2):
    '''This function calculates the  rate of one parameter per another'''
    rate = param1/param2
    return rate

def percentage_rate(param1, param2):
    '''This function calculates the percentage rate of any event'''
    percentage_rate = (param1/param2)*100
    return percentage_rate

def percent_diff(param1, param2):
    '''This function calculates the percentage difference in rates of any event'''
    percent_diff = ((param1-param2)/param2)*100
    return percent_diff


In [ ]:
# Compute rates for all accidents to compare analysis results with
days= 365
hours= 8760
weeks = 52
sum_accidents = len(Accidents)
daily_rate = rate(sum_accidents, days)
hourly_rate = rate(sum_accidents, hours)
weekly_rate = rate(sum_accidents, weeks)
print(f'The daily rate of accidents according to our data is {round(daily_rate)} per day')
print(f'The hourly rate of accidents according to our data is {round(hourly_rate)} per hour')
print(f'The weekly rate of accidents according to our data is {round(weekly_rate)} per week')

In [ ]:
def Get_stats(data, columns):
    '''
    This function returns relevant desriptive statistics of a dataframe column,
    Frequency statistics best describe categorical data.
    The function accepts the dataframe and relevant columns as parameters, 
    Pandas dataframe methods are applied to get the value_counts and column descriptions. 
    It prints out various value counts and returns the summary statistics.
    '''
    cols = data[columns] 
    for i in cols.columns : # Loop through  columns
        
        summary_stats = cols.describe()
        mode_ = cols[i].mode()[0] # calculate the modal value of the features
        percent_mode = percentage_rate(cols[i].value_counts()[mode_], len(cols[i])) # calculate the modal percentage

        print(f'modal value for the {i} column is {mode_}') # Print the modal score of the feature
        print(f' The frequency of ({mode_}) is : ({cols[i].value_counts()[mode_]})') # To get the frequency of the modal class
        print(f'{mode_} makes up {percent_mode}% of the total of the supplied data ') # Print the modal percentage
        print(summary_stats)

## Hours of  day 

 An inspection of the visualization of Accidents frequency per hour of day may confirm our introspection for the time periods of interest. 


In [ ]:
''' Plotting frequency graph of the feature of interest'''

def plot_freq(data, x):
    #define figure size
    plt.figure(figsize=(8,6))
    ax = sns.countplot(data= data, x=x, color = 'teal')
    if x=='Hour':
        ax.set_xlabel("Hour of Day")
        ax.set_xticklabels([str(i) for i in range(1,25)], size=14, rotation= 90)
        
    elif x=='Day_of_Week':
        ax.set_xlabel("Day_of_Week")
        ax.set_xticklabels(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], size=14, rotation=-45)
    
    elif x=='week':
        ax.set_xlabel("week")        
    return ax


In [ ]:
#Visualize dristribution of road accidents per hour of day
all_hrs =plot_freq(Accidents, 'Hour')
plt.title('Frequency of Accidents per Hour of day(2019)')
plt.show()

#list of all figures
figs = []
figs.append(all_hrs)

The plot above reveals highly significant occurences of accidents within the day especially between 7am and 8pm, with major spikes in occurences between 3pm and 6pm, 7am and 8am
The statistical summary of the hours of accident occurence may reveal more about our question

In [ ]:
Get_stats(Accidents, ['Hour']) # Get Summary Statistics of the feature

In [ ]:
#calculate and compare rate of accidents in the modal hour with hourly rate for all accidents
mode_count = Accidents['Hour'].value_counts()[17] # all accidents in the 18th hr

i = rate(mode_count, days) # get rate for 18th in the whole year

i_percent = percent_diff(i, hourly_rate) #compute percentage difference from hourly rate of all accidents

print(i, i_percent)

def avg_percent(col, mode, param1, param2):
    '''calculate rate and percentage change in rates'''
    
    mode_count = col.value_counts()[mode] # accidents for the specified mode

    i = rate(mode_count, param1) # get rate for mode against certain parameter(days or hours)

    i_percent = percent_diff(i, param2) #compute percentage difference between rate of mode and an existing rate

    print(f'average: {i}, percentage:{i_percent}')

In [ ]:
#calculate and compare rate of accidents in the modal hour with hourly rate for all 
avg_percent(Accidents['Hour'], 17, days, hourly_rate)

From Descriptive statistics and simple analysis we infer that,

Accidents happen the least at Night specifically between (20:00 and 06:00) with more occurrences during the day time.

Accidents that occur within the 18th hour of the day(17:00 - 18:00) are greater than the hourly average (13/hr) by 108% (27.95/hr)

Accidents are most frequent at peculiar times in the morning and evening which may be explained by our initial hypothesis. The hours of highest occurs sit well between our proposed "rush hour" where traffic is busiest as a result of socio-economic activities. 

## Days of week 

As socio-economic activities happen during the business week(mon-friday), we can naturally assume that there's less traffic during weekends and inherently less occurences of accidents. 

We review the frequency statistics of accidents across all days to appropriately infer for Accidents in 2019. The related feature has seven unique values in the range 1 to 7 equivalent to the respective days in a week. i.e Sunday to Saturday

In [ ]:
# For days of the week to get days traffic accidents are most likely.
all_days = plot_freq(Accidents,'Day_of_Week')
plt.title('Frequency of Accidents per Day of the week (2019)')
plt.show()
figs.append(all_days)

A quick look at the plot reveals a confirmation of our initial hypothesis. More occurences are recorded during the business week with Friday being the most common day for accidents to happen and a subsequent but not a dramatic decrease in the number of occurences on the weekends

In [ ]:
Get_stats(Accidents, ['Day_of_Week'])

In [ ]:
#from fractions import Fraction
#compute ratio of all occurences on a random weekday to a random weekend day
#saturday = Analysis['Day_of_Week'].value_counts()[7]
#tuesday = Analysis['Day_of_Week'].value_counts()[2]
#ratio = saturday/tuesday
#print(f'The ratio of accidents on a random weekday to a random day of the weekend is {Fraction(ratio).limit_denominator()}')
      
#calculate and compare rate of accidents on the modal day with daily rate for all accidents
avg_percent(Accidents['Day_of_Week'], 6, weeks, daily_rate)

#### Conclusion

The ascending trend of occurences across the week reveals that more accidents happen as the business week progresses. starting on Monday (day 2), through to, and peaks on Friday (day 6).

The modal day of road accidents is Friday, equivalent to the class value 6 of the feature.
The average of accidents that occur on Friday is greater than the daily average (322/day) by 13.8% (366/friday)

We may attribute this to busier commute, a mixture of people commuting from work or school and people going out or traveling for the weekend.

The days of the weekend, Sunday and Monday (day 7 and 1) witness relatively less occurrences which may be attributed to less traffic as a result of fewer activities that increase the use of major roads.


## B. Significant periods of Motorbike accidents

>following similar approach (frequentist statistics) to the question of all road accidents, we investigate the significant periods of motorbike accidents.

### For motorbikes, are there significant hours of the day, and days of the week, on which accidents occur?

In [ ]:
def Convert(data, feature, classes) :
    """
Function to map values of a dataframe column to their code.
It accepts a dataframe, column name and the defined class of changes 
    """
    for k,v in classes.items() : # loop through the dictionary
        data[feature].replace(k,v,inplace=True) # replacement
    return data[feature] # Return feature

In [ ]:
'''On information provided in the variable look up, we map appropriate vehicle types to a motorcycle class to aid analysis'''

# mapping appropriate values from the Vehicle_Type feature to Motorcycle
classes = {2:'Motorcycle (50cc & less)',3:'Motorcycle (51-125cc)',4:'Motorcycle(126-500cc)',
          5:'Motorcycle(501cc+)',22:'Mobility scooter',23:'Electric Motorcycle',
          97:'unknown Motorcycle'}

#Convert appropriate vehicle types to motorcycle
Analysis['Vehicle_Type'] = Convert(Analysis,'Vehicle_Type',classes) # call designated function

Analysis.Vehicle_Type.unique()

In [ ]:
'''We create a dataframe for the Motorcycle class 
by filtering through values in Vehicle_Type feature equal to Motorcycle in the Analysis dataframe'''
m_values =[v for v in classes.values()]
#get index of accidents involving motorcycles
bike_indx = Analysis['Accident_Index'][Analysis['Vehicle_Type'].isin(m_values)]

# subset according to derived index to get dataframe of bike accidents
Motorcycle = Accidents.loc[Accidents['Accident_Index'].isin(bike_indx)]
len(Motorcycle)

In [ ]:
# percentage rate of accidents that involve motorcycles

bike_rates = percentage_rate(len(Motorcycle), len(Accidents))

print('Accidents involving motorcycles make up {}% of road accidents in the UK in 2019'.format(int(bike_rates)))

## Hours of day

In [ ]:
m_hr =plot_freq(Motorcycle,'Hour')
plt.title('Frequency of Motorcycle accidents per hour')
plt.show()
figs.append(m_hr)

In [ ]:
Get_stats(Motorcycle, ['Hour'])

In [ ]:
# Compute rates for all bike accidents to compare analysis results with
days= 365
hours= 8760
weeks = 52
sum_bike_accidents = len(Motorcycle)
daily_bike_rate = rate(sum_bike_accidents, days)
hourly_bike_rate = rate(sum_bike_accidents, hours)
weekly_bike_rate = rate(sum_bike_accidents, weeks)
print(f'The daily rate of bike accidents according to our data is {round(daily_bike_rate)} per day')
print(f'The hourly rate of bike accidents according to our data is {round(hourly_bike_rate)} per hour')
print(f'The weekly rate of bike accidents according to our data is {round(weekly_bike_rate)} per week')

In [ ]:
#calculate and compare rate of bike accidents in the modal hour with hourly rate for all bike accidents
avg_percent(Motorcycle['Hour'], 17, days, hourly_bike_rate)

## Inference

From the descriptive statistics and, simple analysis,

Motorcycle accidents make up 12% of all accidents of all accidents in the UK in 2019

With the 18th hour at the peak, the modal hour has a frequency of 1357 which constitutes 9.60% weekly motorcycle accidents in 2019. 

motorcycle ccidents that occur within the 18th hour of the day(17:00 - 18:00) are greater than the hourly average (2/hr) by 130.5% (3.7/hr)

From the plot, the occurences appear to be seasonal with spikes at certain hours in the day and very drastic reductions at other times. 

There's a sudden increase in occurences between 8am and 10am followed by a sharp decrease within the next hour. From then on, there's a observed gradual but steady increase through the rest of the day up until the end of rush hour when occurences begin to gradually decline up to less than 250 occurences during the night and early mornings.


Most motorcycle accidents happen during the daytime (between 6am and 7pm) with the highest occurences at consistent times with all road accidents. 

We may accrue the observations to our 'rush hour' theory and a concentration of motorcycle use for courier services within cities.


## Days of week

In [ ]:
m_days =plot_freq(Motorcycle,'Day_of_Week')
plt.title('Frequency of Motorcycle accidents per Day of the week')
plt.show()
figs.append(m_days)

In [ ]:
Get_stats(Motorcycle, ['Day_of_Week'])

In [ ]:
#calculate and compare rate of bike accidents on the modal day with daily rate for all bike accidents
avg_percent(Motorcycle['Day_of_Week'], 6, weeks, daily_bike_rate)

### Inference

The ascending trend of occurences across the week reveal that more accidents happen as the business week progresses. starting with Monday (day 2), through to, and peaking on Friday (day 6).

The modal day of road accidents is Friday, equivalent to the class value 6 of the feature. Occurences on Fridays with a frequency of 2332 accidents constituted 16.5% of weekly motorcycle accidents in the UK in 2019. We may attribute this to busier traffic, as a product of the rush hour of the weekend start.


The average of bike accidents that occur on Friday is greater than the daily average (39/day) by 15.8% (44.8/friday)

The rest of the distribution is however almost uniform across the week as all days saw bewtween 2000 and 2500 motorcycle accidents throughout the year. We can conclude that the occurence of these accidents are not dependent on the day of the week and there is no obvious correlation between the days and the accidents. 

Based on our Courier services theory, we consider that courier services like food delivery that make use of motorcycles, carry on regardless of workdays as such services are relatively always in demand. Besides courier services, the largest owners and drivers of motorcycles are young men between the ages 17 and 29 ([National Travel Survey][]).  

These statistical observations are similar to the distribution of all road accident occurences, with the major difference being a consistently high frequency of motorcycle accidents across all days in the week.
    

[National Travel Survey]: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/694965/motorcycle-use-in-england.pdf

# C. Significant periods of accidents involving pedestrians

### For pedestrians, are there significant hours of the day, and days of the week, on which they are more likely to be involved in an accident?

We may start by investigating the assumption that pedestrians are at a higher risk of traffic accidents at night when compared to daytime because of reduced visibility.

In [ ]:
'''On information provided in the variable look up, 
we map appropriate casualty values to a casualty class to aid analysis'''

#Analysis['Casualty_Class'].replace(3,'Pedestrian',inplace=True)
#Analysis['Casualty_Class'].replace(2,'Passenger',inplace=True)
#Analysis['Casualty_Class'].replace(1,'Driver or Rider',inplace=True)

#map Casualty_Class value to equivalent catgorical feature
Casualty = {1:'Driver or rider',2:'Passenger',3:'Pedestrian'}

#Convert appropriate casualty value to casualty category
Analysis['Casualty_Class'] = Convert(Analysis,'Casualty_Class', Casualty)

In [ ]:
'''We create a dataframe for the Pedestrian class 
by filtering through values in the Casualty_Class feature equal to Pedestrian in the Analysis dataframe'''

#get index of accidents involving pedestrians
ped_indx = Analysis['Accident_Index'][Analysis['Casualty_Class']== 'Pedestrian']

# subset according to derived index to get dataframe of bike accidents
Pedestrian = Accidents.loc[Accidents['Accident_Index'].isin(ped_indx)]
len(Pedestrian)

In [ ]:
# Compute rates for all accidents inolving pedestrians to compare analysis results with
days= 365
hours= 8760
weeks = 52
sum_ped_accidents = len(Pedestrian)
daily_ped_rate = rate(sum_ped_accidents, days)
hourly_ped_rate = rate(sum_ped_accidents, hours)
weekly_ped_rate = rate(sum_ped_accidents, weeks)
print(f'The daily rate of accidents involving pedestrians according to our data is {round(daily_ped_rate)} per day')
print(f'The hourly rate of accidents involving pedestrians according to our data is {round(hourly_ped_rate)} per hour')
print(f'The weekly rate of accidents involving pedestrians according to our data is {round(weekly_ped_rate)} per week')

## Hours of day

In [ ]:
p_hr =plot_freq(Pedestrian, 'Hour' )
plt.title('Frequency of Pedestrian accidents per hour')
plt.show()
figs.append(p_hr)

In [ ]:
Get_stats(Pedestrian, ['Hour'])

In [ ]:
#calculate and compare rate of pedestrian accidents in the modal hour with hourly rate for all pedestrian accidents
avg_percent(Pedestrian['Hour'], 15, days, hourly_ped_rate)

From descriptive statistics and, basic analysis,

There are relatively higher occurences of accidents involving pedestrians occuring during the day (between 7am and 7pm) than at other times. 

The distribution shows a surge at 08:00 hours with respect to other periods of the morning. The most significant hours are between 15:00 hours and 18:00 hours. However, unlike with vehicular accidents whose peak values are within the 17th hour, 
the highest occurences of pedestrian accidents happen within the 15th hour of the day. pedestrians are involved in accidents at a rate of 6.5/hr which is 145% higher than the hourly average 3/hr.

The numbers go up to 2364 occurences which constitutes 10.21% of the total pedestrian accidents that happen daily. 

The highest periods of pedestrian traffic correlates with Vehicular traffic as well. We can infer that the same theories apply to both categories of accidents.


Socio-economic activities have a high influence on the concentration of traffic at various times of the day and heavier traffic inversely implies a higher risk of accidents including those involving pedestrians.

## Days of week

In [ ]:
p_days = plot_freq(Pedestrian, 'Day_of_Week')
plt.title('Frequency of Pedestrian accidents per Day of the week')
plt.show()
figs.append(p_days)

In [ ]:
Get_stats(Pedestrian,['Day_of_Week'])

In [ ]:
#calculate and compare rate of pedestrian accidents on the modal day with daily rate for all accidents
avg_percent(Pedestrian['Day_of_Week'], 6, weeks, daily_ped_rate)

Day of Week with regards to pedestrian accident occurences.

The day with the highest occurences of accidents involving pedestrains is Friday, equivalent to the the class with value 6 in the feature. 

Pedestrian accidents on friday go up to 3859 occurences and constituted 16.66% of total accidents per week. 

The average number of accidents involving pedestrians that occur on Fridays is less than the daily average (63/day) by 17% (74/friday)


The distribution of other occurences across the business days of the week (Monday - Friday) is almost evenly distributed. 

Similar to the distribution of all accidents, We can assume that Friday being the transition of the week into weekends and the associated commute is responsible for the increased frequency of accidents on that day.

The days of the weekend, witnesses relatively less accidents because traffic is lighter as a result of less activities requiring the use of major roads.

# D. Impact of daylight savings on road traffic accidents

>Daylight savings In the Uk in 2019 started on the 31st of March, and ended on the 27th of Octobe of 2019. ([time and date.com][])


### What impact does daylight savings have an impact on the occurence of accidents?

If our hypothesis is that Daylight savings have an impact on the accident occurences, The distribution of accidents by defined periods of the year will reveal some obvious trend during the first few days of daylight savings. 


We may test the significance of this assumption.

H0 = Daylight savings has no impact on the occurence of accidents.

H1 = Daylight savings has an impact on the occurence of accidents.


[time and date.com]: https://www.timeanddate.com/time/change/uk/london?year=2019

In [ ]:
def stats_test(data, col, test) :
    """
Function to perform Statistical tests of normality and/or parametric tests on a dataframe.

It accepts a list of dataframes, 
the related data column and,
the specific test to be performed.

There are three tests considered in this function;

ttest == student T-test to compare the mean of two groups in a distribution. 
shapiro == frequency statistic test to determine if data (feature in this case) is normally distributed.
ztest == test of mean of two different normal distributions

    """
    if test.lower() == 'shapiro': # Condition for shapiro
        shapiro_test,pval = shapiro(data[col]) # Apply shapiro 
        print("p-value for significance is: ", pval)# print the p-value
        if pval <0.05:
            return "Reject null hypothesis for shapiro"# if the value less than the threshold P_value
        else:
            return "Accept null hypothesis for shapiro"# if the  P_value is greater than threshold P_value
        
    elif test.lower() == 'ttest' : # Check condition for ttest
        ttest_test,pval = ttest_ind(data[0][col],data[1][col]) # Apply 2-sample ttest
        print("p-value for ttest significance is: ", pval) # print the p-value
        if pval <0.05: # Condition for p_values 
            return "Reject null hypothesis for the T-test" # if the value less than the threshold P_value
        else: 
            return "Accept null hypothesis for the T-test" # if the  P_value is greater than threshold P_value
        
    elif test.lower() == 'ztest':# Condition for ztest
        ztest_test ,pval = stests.ztest(data[0][col], x2=data[1][col], value=0,alternative='two-sided')# Apply z_score
        print("p-value for ztest significance is: ", pval) # print the p-value 
        if pval<0.05:
            return "reject null hypothesis for ztest" # if the value less than the threshold P_value
        else:
            return "accept null hypothesis for ztest" # if the  P_value is greater than threshold P_value
        
    else :
        return "The test you have selected is not available" # Check for wrong choice

In [ ]:
'''
Function performs a non parametric test; Wilcoxon Rank-Sum test (aka mann-withney-u)
mannwhitenyu == nonparametric test of equal probabilities between two independent distributions
parameters are a list of dataframes and the associated column
'''
def non_param(data, col):
    mannwhitneyu_test,pval = mannwhitneyu(data[0][col], data[0][col])# Apply manwhitneyu 
    print("p-value for mannwhitneyu significance is: ", pval) # print the p-value 
    if pval <0.05:
        return("Reject null hypothesis for mannwhitneyu") # if the value less than the threshold P_value
    else:
        return("Accept null hypothesis for mannwhitneyu") # if the  P_value is greater than threshold P_value
            

In [ ]:
'''Visualize the distribution'''

# Plotting for the distribution of accidents across all weeks
acc_wk = sns.histplot(data= Accidents, x ='week', kde = True, Color = 'teal')
plt.title('Distribution of Accidents across all weeks in 2019')
plt.show()
figs.append(acc_wk)

The overall distribution of accidents across all weeks of the year does not appear normal at all. We also observe major spikes in the 1st, Last and the 27th weeks of the year

In [ ]:
'''Visualize the distribution'''

# Plotting for the distribution of accidents across all weeks
acc_day = sns.histplot(data= Accidents, x ='Day_of_Week', kde = True, Color = 'teal')
plt.title('Distribution of Accidents by day of week in 2019')
plt.show()
figs.append(acc_day)

### 1week into daylight savings

Initial analysis is based on observed difference in occurences the week immediately after the start of daylight savings and the rest of the year excluding the week immediately Daylight savings ends.

We test our hypotheses by comparing the distribution of occurences across days in both time periods.

however, we must confirm that both distributions data meet all statistical requirements including normality in order to perform a parametric test of comparison, otherwise we select a non-parametric test.

Tests included in the experiment are because the distributions to be compared have quantitative and discrete outcomes (count of accidents) and are from independent distributions (different periods).

For the statistical tests, p-value is set to 5% confidence. If the test score is greater than or equal to 5%, we accept the null hypothesis. 

Accepting the null hypothesis implies that there is no special relationship between the distributions we are comparing.

For the purpose of our analysis, we define the weeks immediately after the start and end of the daylight savings

In [ ]:
FORMAT = "%Y-%m-%d"
start_date=datetime.datetime.strptime("2019-03-31", FORMAT) # start of daylight savings

end_date=datetime.datetime.strptime("2019-10-27", FORMAT) # end of daylight savings

day7_after_start = start_date + datetime.timedelta(weeks=1) # a week from the start date

day7_after_end = end_date + datetime.timedelta(weeks=1) # a week from the end date


# define all three periods 

week_after_start = Accidents.loc[(Accidents['Date'] > start_date) & (Accidents['Date'] <= day7_after_start)]

week_after_end = Accidents.loc[(Accidents['Date'] > end_date) & (Accidents['Date'] <= day7_after_end)]


other_weeks1 = Accidents.loc[(~Accidents['Date'].isin(week_after_start)) & (~Accidents['Date'].isin(week_after_end))]


In [ ]:

'''get the occurences within both periods for comparison.'''

# The Dataframe of accidents per day the week after Daylight Savings started 
week_after = week_after_start.groupby('Day_of_Week').size().reset_index(name='Accidents')

# The Dataframe of accidents per day for other weeks in the year 
other_weeks = other_weeks1.groupby('Day_of_Week').size().reset_index(name='Accidents')
week_after.head()

In [ ]:
other_weeks

In [ ]:
new_data1 =pd.DataFrame({'Day_of_Week':[1, 4], 'Accidents':[0, 0]})
week_after = week_after.append(new_data1, ignore_index = True)# include days with no incidence

In [ ]:
week_after

In [ ]:
'''We visualize the distribution of accidents in both periods for observed differences'''

# Put Sunday first because that is when the time changes
af_ds = sns.barplot(data= week_after, x='Day_of_Week', y='Accidents', color = 'navy')
#ax = sns.histplot(data= week_after, x ='Day_of_Week', kde = True, Color = 'teal')

af_ds.set_xticklabels(["Sunday (DST)", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], size=14, rotation= 90)
plt.title('Accidents in the week after the start of Daylight Savings in the UK(2019))', y=1.03, size=10)
plt.show()
figs.append(af_ds)

In [ ]:
other_wks = sns.barplot(data= other_weeks, x='Day_of_Week', y='Accidents', color= 'black')
other_wks.set_xticklabels(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], size=14, rotation= 90)
plt.title('distribution density of accidents by day across other weeks of the year in the UK(2019))', y=1.03, size=10)
plt.show()
figs.append(other_wks)

In [ ]:
# function to test normality of distribution and compare distribution of accidents within both weeks
def compare_dists(dfs, col):

  # confirm normality
    outcome = []
    for df in dfs:
        outcome.append(stats_test(df, col, 'shapiro'))
    print(outcome)
    if outcome[0] == 'we accept null hypothesis for shapiro' and outcome[1] == 'we accept null hypothesis for shapiro':
        print ('Both distributions are Gaussian')
        text=input('select a parametric test')# ask to define parametric test
        # Now testing hypothesis
        result = non_param(dfs,col,text)
        return result
    else: # if at least one distribution is not normal
        print('one or both of the distributions are not Gaussian, running Wilcoxon Rank-Sum test ') 
        #text=input('select a non-parametric test') # ask to select a non_parametric test
        # now testing hypothesis
        result = non_param(dfs, col)
        return result 

In [ ]:
compare_dists([week_after, other_weeks], 'Accidents') #hypothesis testing

### Inference

The P-value is greater than 0.05, we therefore accept the null hypothesis and infer that Daylight savings has no impact on the occurence of accidents in the week immediately after it starts.

## 1week after daylight savings

We compare and repeat the hypothesis test for the week after daylight savings ends and other weeks of the year excluding the weeks immediately after daylight savings ended.

In [ ]:
'''get the occurences within both periods for comparison.'''

# The Dataframe of accidents per day the week after Daylight Savings ended
week2_after = week_after_end.groupby('Day_of_Week').size().reset_index(name='Accidents')

week2_after

In [ ]:
new_data2 = pd.DataFrame({'Day_of_Week':[1, 7],'Accidents':[0, 0]})
week2_after = week2_after.append(new_data2, ignore_index = True)

In [ ]:
'''We visualize the distribution of accidents in both periods for observed differences'''

# Put Sunday first because that is when the time changes
af_wk2 = sns.barplot(data= week2_after, x='Day_of_Week', y='Accidents', color= 'navy')
af_wk2.set_xticklabels(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], size=14, rotation= 90)
plt.title('Accidents in the week after the end of Daylight Savings in the UK(2019))', y=1.03, size=10)
plt.show()
figs.append(af_wk2)

In [ ]:
compare_dists([week2_after, other_weeks], 'Accidents')

We also confirm that there is no significant difference in the distribution of accidents in the week after daylight savings ended.

# Further investigation 

1. "Chi-by-eye"
2. Descriptive statistics


In [ ]:
Get_stats(week_after_start, ['Day_of_Week'])

In [ ]:
Get_stats(week_after_end, ['Day_of_Week'])

In [ ]:
Get_stats(other_weeks1, ['Day_of_Week'])

In [ ]:
print(f'accidents occured at the rate of {round(rate(len(week_after_start),7))} per day in the week after DS started' )

print(f'accidents occured at the rate of {round(rate(len(week_after_end),7))} per day in the week after DS ended')

print(f'accidents occured at the rate of {round(rate(len(Accidents), 365))} per day in all other weeks of the year')

## Inference

from the statistical tests; 

**Shapiro**
We accept H1 and confirm that all distributions are not normal and therefore proceed with non-parametric tests

**Manwithneyu**
We also ascertain the confidence of probability between occurences in both periods confirming no difference in observations after the start and end of daylight savings.



Comparing descriptive statistics of the distributions;

The impact of daylight savings on the frequency of accidents is insignificant.

2273 accidents happened in the week immediately after daylight savings compared to the weekly average of 2260 for the whole of the year with 32.4% of those happening on Thursdays. The average per day was also 325 accidents compared to 322 accidents per day in the year. 

In the week after daylight savings ended, a total of 2275 accidents occured compared to the 2260 for all weeks in the year with 41.6% of them happening on Tuesdays. The average per day was 325 accidents compared to 322 across other weeks. 


## E. Impact of sunrise and sunset time on road accidents

### What impact, if any, does sunrise and sunset times have on road traffic accidents?

>since sunrise and sunset times vary across the year in the uk. we will investigate by group months of the year.
The darkest months(with shorter daylight hours)are January, February, October, November and December with less than 11 hours of daylight on the average. The longest days are between March and September.

long_days = Mar - sept

short_days = Oct - Feb

According to ([world data(2019)][])

sunrise across the UK during long_days was between 7:12AM(**7.12**) hours and 7:22AM(**7.22**)hrs. 

sunrise across the UK during short_days was between 4:40AM(**4.40**) hours and 6:33AM(**6.33**)hrs.

sunset across the UK during long_days was between 6:06PM(**18.06**) hours and 9:21PM(**21.21**)hrs.

sunset across the UK during short_days was between 3:53PM(**15.53**) hours and 6:09PM(**18.09**)hrs.


Using similar methods as defined before, we investigate the impact of sunrise and sunset on the occurences of accidents by comparing daylight hours with these periods

This is the basis of performing frequentist statistics and creation of dataframes.


[world data(2019)]: https://www.worlddata.info/europe/united-kingdom/sunset.php

#### Hypothesis testing
For:
The darkest months

The brightest months

H0 = Sunrise and sunset have no impact on the occurence of accidents

H1 = Sunrise and sunset have an impact on the occurence of accidents

In [ ]:
# define both periods

brighter_months = Accidents.loc[(Accidents.Month >2) | (Accidents.Month <= 9)]

dark_months = Accidents.loc[(Accidents.Month >9) | (Accidents.Month <= 2)]


### During the darkest months

In [ ]:
#sunrise and sun set times in darkest months
rise_dark_months = dark_months.loc[(dark_months.decimal_time >=4.40) & (dark_months.decimal_time <=6.33 )]
set_dark_months =dark_months.loc[(dark_months.decimal_time >=15.53) & (dark_months.decimal_time <=18.09 )]

#normal daylight period
day_time_dm = dark_months.loc[(dark_months.decimal_time >6.33) & (dark_months.decimal_time <15.53)]

'''Create dataframes'''

# accidents per hour during sunrise in the darkest months
sunrise_dm = rise_dark_months.groupby('decimal_time').size().reset_index(name='Accidents')

# accidents per hour during sunset in the dark months
sunset_dm = set_dark_months.groupby('decimal_time').size().reset_index(name='Accidents')

# accidents per hour during sunset in the dark months
daytime_dm = day_time_dm.groupby('decimal_time').size().reset_index(name='Accidents')

In [ ]:
compare_dists([daytime_dm, sunrise_dm], 'Accidents') #for sunrise in brigher months

In [ ]:
compare_dists([daytime_dm, sunset_dm], 'Accidents') #for sunset in brigher months

In [ ]:
sunrise_dm.describe()

In [ ]:
#percentage rate of accidents during sunrise in the dark months
percentage_rate(len(rise_dark_months), len(dark_months))

In [ ]:
#rate per hour during sunrise. There are exactly 1.88hrs (giga calculator) between 4.40 and 6.33am= 1.88 x 151 (no of days in the dark months)
rate(len(rise_dark_months), (1.88*151))

In [ ]:
sunset_dm.describe()

In [ ]:
#percentage rate of accidents during sunset in the dark months
percentage_rate(len(set_dark_months), len(dark_months))

In [ ]:
#rate per hour during sunset. There are exactly 2.27hrs(giga calculator) between 15.53 and 18.09pm= 2.27 x 151 (no of days in the dark months)
rate(len(set_dark_months), (2.27*151))

In [ ]:
daytime_dm.describe()

In [ ]:
#percentage rate of accidents during normal daylight hours in the dark months
percentage_rate(len(day_time_dm), len(dark_months))

In [ ]:
#rate per hour during normal daylight hours. 
#There are exactly 9.33 hours(giga calculator) between 6.33am and 15.53pm= 9.33 x 151 (no of days in the dark months)
rate(len(day_time_dm), (9.33*151))

In [ ]:
plt.figure(figsize = (10, 4))
dark = sns.kdeplot(data= dark_months, x='decimal_time')
plt.axvline(x=4.40, linewidth=1.5, color="red", linestyle="dashed")
plt.axvline(x=6.33, linewidth=1.5, color="green", linestyle="dashed")
plt.axvline(x=15.53, linewidth=1.5, color="blue", linestyle="dashed")
plt.axvline(x=18.09, linewidth=1.5, color="0.25", linestyle="dashed")
plt.title('Distribution of Accidents around sunrise and sunset in the darkest months ', fontdict={'size':15})
plt.xlabel('Time of Day')
plt.show()
figs.append(dark)

### During the brightest months

In [ ]:
#sunrise and sun set times in brighter months
rise_brighter_months = brighter_months.loc[(brighter_months.decimal_time >=7.12) & (brighter_months.decimal_time <=7.22 )]
set_brighter_months = brighter_months.loc[(brighter_months.decimal_time >=18.06) & (brighter_months.decimal_time <=21.21 )]


#normal daylight period
day_time_bm = brighter_months.loc[(brighter_months.decimal_time >7.22) & (brighter_months.decimal_time <18.06)]


'''Create dataframes'''

# accidents per hour during sunrise in the bright months
sunrise_bm = rise_brighter_months.groupby('decimal_time').size().reset_index(name='Accidents')


# accidents per hour during sunset in the bright months
sunset_bm = set_brighter_months.groupby('decimal_time').size().reset_index(name='Accidents')


# accidents per hour during sunset in the bright months
daytime_bm = day_time_bm.groupby('decimal_time').size().reset_index(name='Accidents')

In [ ]:
compare_dists([sunset_bm, daytime_bm], 'Accidents') #for sunset in the brightest months

In [ ]:
compare_dists([sunrise_bm, daytime_bm], 'Accidents') #for sunrise in the brightest months

# Further investigation 

1. Descriptive statistics
2. "Chi-by-eye"

In [ ]:
sunrise_bm.describe()

In [ ]:
#percentage rate of accidents during sunrise in the brighter months
percentage_rate(len(rise_brighter_months), len(brighter_months))

In [ ]:
#rate per hour during sunrise. There are exactly 0.17hrs (giga calculator) between 7.12 and 7.22am= 0.17 x 214 (no of days in the brighter months)
rate(len(rise_brighter_months), (0.17*214))

In [ ]:
sunset_bm.describe()

In [ ]:
#percentage rate of accidents during sunset in the brighter months
percentage_rate(len(set_brighter_months), len(brighter_months))

In [ ]:
#rate per hour during sunset. There are exactly 3.25hrs(giga calculator) between 18.06 and 21.21pm= 3.25 x 214 (no of days in the dark months)
rate(len(set_brighter_months), (3.25*214))

In [ ]:
daytime_bm.describe()

In [ ]:
#percentage rate of accidents during normal_daylight hours in the dark months
percentage_rate(len(day_time_bm), len(brighter_months))

In [ ]:
#rate per hour during normal daylight hours. 
#There are exactly 10.73 hours(giga calculator) between 7.22am and 18.06pm= 10.73 x 214 (no of days in the dark months)
rate(len(day_time_bm), (10.73*214))

In [ ]:
plt.figure(figsize = (10, 4))
bright = sns.kdeplot(data= brighter_months, x='decimal_time')

plt.axvline(x=7.12, linewidth=1.5, color="red", linestyle="dashed")
plt.axvline(x=7.22, linewidth=1.5, color="green", linestyle="dashed")
plt.axvline(x=18.06, linewidth=1.5, color="blue", linestyle="dashed")
plt.axvline(x=21.21, linewidth=1.5, color="0.25", linestyle="dashed")
plt.title('Distribution of Accidents around sunrise and sunset in the brightest months ', fontdict={'size':15})
plt.xlabel('Time of day')
plt.show()
figs.append(bright)

## Inference

Sunrise and sunset have an impact on the occurence of accidents. 


It is important to note that although sunrise and sunset times vary across months in the year, the distribution of occurences across hours of the day remains consistent for both periods(darkest and brightest months) as is observable from the plots.


# F. Relationship of vehicle characteristics and occurences of Accidents.

### Are there particular types of vehicles (engine capacity, age of vehicle, etc.) that are more frequently involved in road traffic accidents?

1. Descriptive statistics
2. Chi_by_eye

In [ ]:
 #################################### Quantitative features #######################################################
Get_stats(Analysis,['Age_of_Vehicle','Engine_Capacity_(CC)'])

In [ ]:
veh_features = Analysis[['Age_of_Vehicle','Engine_Capacity_(CC)']]

In [ ]:
for i in veh_features.columns:
    #display boxplots
    #sns.set_theme(style="white")
    sns.boxplot(data=veh_features, x=i, color = 'brown')
    plt.title(f'distribution of accidents in vehicles based on the {i}')
    plt.show()

The boxplot shows the presence of outliers in the distribution of accidents accorcing to Vehicle engine capacity. 
Outliers must be handled to make meaningful statistical deductions about the data

In [ ]:
def Fix_outliers(data, feature, method, upper = None, lower = None, impute = None):
    """
        Function handles outliers using various methods. 
        It accepts a dataframe, a dataframe column, a method, 
        an upper percentile score- none by default,
        a lower percentile score - none by default
        impute - none by default 
        as parameters. 

        There are three methods defined in this function :
        Imputation = replacing outliers with a constant value from the distribution
        Capping = replacing outliers with the upper and lower percentile values
        Log_transform = transforming the column values to logrithmic values
    """
    if method.lower() == 'imputation': #Checking condition for imputation
        Q1 = data[feature].quantile(lower) # Storing value for upper limit
        Q3 = data[feature].quantile(upper) # Storing values for lower limit
        print(data[feature].skew()) # Printing the skewness of the feature
        data[feature] = np.where(data[feature] < Q1, impute, data[feature] ) #Replace outliers with an imputted value and assign it to the feature,lower limit
        data[feature] = np.where(data[feature] > Q3, impute, data[feature] )# Replace outlierswith an imputted and assign it to feature, upper limit
        print(df[feature].skew())# Printing the skewness of the feature afterwards
        return data[feature] # Return the fixed feature
    elif method.lower() == 'capping': # Check condition for capping
        Q1 = data[feature].quantile(lower) #Assign value for lower limit
        Q3 = data[feature].quantile(upper) # Assign value for upper limit
        print(data[feature].skew()) # Print the skewness of the feature
        data[feature] = np.where(data[feature] < Q1, Q1, data[feature] ) #Replace outliers with the lower limit value and assign it to the feature
        data[feature] = np.where(data[feature] > Q3, Q3, data[feature] ) # Replace outliers with the upper limit value and assign it to the feature
        print(data[feature].skew()) # Print the skewness of the feature
        return data[feature] #Return the fixed feature
    elif method.lower() == 'log_transform': # Checking condition for log_transform
        print(data[feature].skew()) #Print skewness of the feature
        data[feature] = np.log(data[feature]) + 1 # Performing the log transformation
        print(data[feature].skew()) # print the skewness after fixing
        return data[feature] # Return the feature

In [ ]:
veh_features['Age_of_Vehicle'] = Fix_outliers(veh_features, 'Age_of_Vehicle', 'capping', 0.95, 0.05)

veh_features['Age_of_Vehicle'] = Fix_outliers(veh_features, 'Engine_Capacity_(CC)', 'capping', 0.95, 0.05)

In [ ]:
for i in veh_features.columns:
    sns.boxplot(data=veh_features, x=i, color = 'brown')
    plt.show()

In [ ]:
Analysis.Age_of_Vehicle.max()

Investigating the frequency of accidents based on invidual features.

In [ ]:
######################################## Age of Vehicle #####################################################
# Bin age of vehicle to simplify analysis and generate column
Analysis['Age_of_Vehicle'] = pd.cut(x=df['Age_of_Vehicle']
                         , bins=[-1, 0, 5, 10,  20, 40, 70, 94 ]
                         , labels=['unknown','1-5','6-10','11-20','21-40','40-70','70+'], include_lowest=True)

Age_of_Vehicle = Analysis.groupby('Age_of_Vehicle').size().reset_index(name='Accidents')


In [ ]:
#visualize distribution
fig, ax = plt.subplots(figsize=(8, 5))
#sns.set_theme(style="white")


Age_of_Vehicle.plot(x = 'Age_of_Vehicle', kind='barh', ax=ax, cmap='PRGn_r')
ax.set_title('\nDistribution of accidents based on Age of vehicle\n', fontsize=14, fontweight='bold')
ax.set(xlabel='frequency', ylabel='')
ax.legend(bbox_to_anchor=(1.25, 0.98), frameon=False)

sns.despine(top=True, right=True, left=True, bottom=True);
figs.append(fig)

In [ ]:
Age_of_Vehicle['%'] = Age_of_Vehicle[["Accidents"]].apply(lambda x: 100*x/x.sum())
Age_of_Vehicle

'The size – or cubic capacity – of a Vehicle’s engine is measured in cubic centimetres (cc). It refers to the amount of air and fuel that can be pushed through the cylinders in the engine. In most cases, the general rule of thumb is that the bigger the capacity, the more powerful it tends to be' [Cinch][]. 

Safe limits for motorbikes are between 200 & 500CC and the recommended maximum capacity for any vehicle that is at least 1100kg and naturally aspirated is its Original mass (kg) x 5.0 [Street Machine][]. The minimum for the lowest motorcycle category test is 120cc [ndirect.gov.uk][] and for other vehicles is 1000 cc [MCNally Institute][] on the average. 

Based on this knowledge, we may group engine capacities by power for easier analysis setting the treshold at 7500CCs for vehicles other than motorcycles

[Cinch]: https://www.cinch.co.uk/jargon/cc-cubic-capacity#:~:text=The%20size%20%E2%80%93%20or%20cubic%20capacity,powerful%20it%20tends%20to%20be.
[Street Machine]: https://www.whichcar.com.au/features/diy/how-to-work-out-the-largest-road-legal-engine-size-for-your-street-machine
[ndirect.gov.uk]: https://www.nidirect.gov.uk/articles/minimum-requirements-test-vehicles
[MCNally Institute]: https://www.mcnallyinstitute.com/what-percentage-of-the-uk-are-over-3l-engine-cars/

In [ ]:
######################################## Engine capacity #####################################################
#map other vehicle types.
vehicles = {1:'Pedal cycle',8:'Taxi/Private car',9:'car',10:'Minibus',
             11:'Bus/Coach',16:'Ridden horse',17:'Agric Vehicle',
             18:'Tram',19:'Van/Goods 3.5t',
             21:'Goods 7.5 tonnes mgw and over',20:'Goods over 3.5t. and under 7.5t',
             90:'Other vehicle', 98:'other Goods Vehicle',-1:'missing'}
Analysis['Vehicle_Type'] = Convert(Analysis,'Vehicle_Type', vehicles)

In [ ]:

#for motorcycles
#update dataframe
Motorcycle = Analysis.loc[Analysis['Vehicle_Type'].isin(m_values)]

#bin engine capacity for motorcycles 
Motorcycle['Engine_Capacity_range'] = pd.cut(x=Motorcycle['Engine_Capacity_(CC)']
                         , bins=[-1, 0, 50, 610, 7817 ]
                         , labels=['unknown','low','good range','very high'], include_lowest=True)


# for non motorcycles 
non_two_wheeled = Analysis.loc[~(Analysis['Vehicle_Type'].isin(m_values))]

#bin engine capacity for other vehicles
non_two_wheeled['Engine_Capacity_range'] = pd.cut(x=Analysis['Engine_Capacity_(CC)']
                         , bins=[-1, 0, 1000, 7500, 29980 ]
                         , labels=['unknown','low','good range','very high'], include_lowest=True)



#vehicle_cc = non_two_wheeled.groupby('Engine_Capacity_range').size().reset_index(name='Accidents')

#Motorcycle_cc = Motorcycle.groupby('Engine_Capacity_range').size().reset_index(name='Accidents')

# define all vehicle types excluding motorcycles
non_two_wheeled['Vehicle_cat'] = non_two_wheeled['Vehicle_Type']

#cast all motorcycles to class 'motorcycle'
Motorcycle['Vehicle_cat'] = 'Motorcycle'

#get all vehicles
all_vehicles = pd.concat([non_two_wheeled, Motorcycle])

        
all_vehicles['Vehicle_cat'].unique()

In [ ]:
#define figure size
plt.figure(figsize=(6,5))
#sns.set_theme(style="white")
#display countplot
all_veh = sns.countplot(data = all_vehicles
            ,x = 'Engine_Capacity_range'
            #,hue = 'Vehicle_cat'
            ,color='orange'
            )
plt.xlabel('Engine_capacity_range', size=16)
plt.ylabel('frequency', size=16)
plt.title('Distribution of Accidents based on vehicle engine capacity', size=18)
plt.show()
figs.append(all_veh)

In [ ]:
percentage_rate(len(all_vehicles[all_vehicles['Engine_Capacity_range']=='unknown']), len(all_vehicles))

From the analysis, Vehicles with ok engine capacity account for the highests occurences of accidents. Vehicles with engine capacity in unknown are the second largest group involved in accidents. Vehicles whose engine capacities are unknown also account for about 21.8% of all vehicles invoved in accidents in 2019.  


In [ ]:
########################################## Propulsion code ############################################

# map values to appropriate vehicle propulsion class

propulsion = {1:'Petrol',2:'Heavy',3:'Electric',4:'Steam',
             5:'Gas',6:'Petrol/Gas(LPG)',7:'Gas/Bi-fuel',
             8:'Hybrid electric',9:'Gas Diesel',
             10:'New fuel technology',11:'Fuel cells',
             12:'Electric diesel','M':'Undefined',-1:'missing'}
Analysis['Propulsion_Code'] = Convert(Analysis,'Propulsion_Code', propulsion)

In [ ]:
Get_stats(Analysis, ['Propulsion_Code'])

In [ ]:
#plot the distribution of accidents according to vehicle propulsion codes
prop_code = sns.countplot(data= Analysis, y='Propulsion_Code', color ='teal')
plt.title("distribution of accidents by vehicle propulsion codes", size = 14)
plt.tick_params(axis='x', rotation=90)
plt.show()
figs.append(prop_code)

The analysis carried out reveals that vehicles whose propulsion code is Petrol are the most likely to get involved in accidents with their such vehicles involved in 45.41% of all accidents.

*NB.* The available data indicates that there is a problem with identifying and collecting vehicle feature information for vehicles involved in road traffic accidents.

# G. Relationship between Environmental conditions and occurence of Accidents

### Are there particular conditions (weather, geographic location, situations) that generate more road traffic accidents?

It would seem a fair assumption that driving in poor weather conditions and low light may increase the risk of accidents. 
We investigate to confirm this assumption


### Weather conditions

In [ ]:
################################ Investigating accidents during varying weather ################################## 
# Convert feature values to weather conditions
weather = {1:'Fine no high winds',2:'Raining no high winds',3:'Snowing no high winds',4:'Fine + high winds',
             5:'Raining + high winds',6:'Snowing + high winds',7:'Fog or mist',
             8:'Other',9:'Unknown',
             -1:'missing'}
Accidents['Weather_Conditions'] = Convert(Accidents,'Weather_Conditions', weather)

In [ ]:
#define figure size
plt.figure(figsize=(6,6))
#sns.set_theme(style= 'white')
#display countplot
w_cond = sns.countplot(y="Weather_Conditions", data=Accidents, palette="PRGn_r")
plt.ylabel('Weather_Condition', size=16)
plt.xlabel('no of accidents', size=16)
plt.title('Accidents during various weather conditions', size=18)
figs.append(w_cond)

In [ ]:
# accidents by weather conditions
weather = Accidents.groupby('Weather_Conditions').size().reset_index(name='Accidents')
Get_stats(Accidents, ['Weather_Conditions'])

In [ ]:
#visualize distribution
fig, ax = plt.subplots(figsize=(6, 6))

weather.plot(x = 'Weather_Conditions', kind='barh', ax=ax, cmap='PRGn_r')
ax.set_title('\nDistribution of accidents based on Age of vehicle\n', fontsize=14, fontweight='bold')
ax.set(xlabel='frequency', ylabel='')
ax.legend(bbox_to_anchor=(1.25, 0.98), frameon=False)

sns.despine(top=True, right=True, left=True, bottom=True);

From the summary statistics and plot of the distribution we can deduce that most accidents occurred when the weather was fine with no high winds.
With the frequency during this period at 92316 accidents accounting for 78.5% of all accidents.

We can infer that bad weather conditions have little to no impact on the occurence of accidents.

### Geographic Location

In [ ]:
############################# Investigating accidents based on Geographic Location ######################################
# read in file containing Local Authority district names for matching.
LA = pd.read_csv('LA.csv')
LA.info()

In [ ]:
LAs = dict(zip(LA.code, LA.label))
LA['label'].nunique()

In [ ]:
Accidents['Local_Authority_(District)'].nunique()

In [ ]:
#match LA class
Accidents['LA'] = Convert(Accidents,'Local_Authority_(District)', LAs)
Analysis['LA']= Convert(Analysis,'Local_Authority_(District)', LAs)
Accidents['LA'].nunique()

In [ ]:
coord = Accidents.groupby(['Longitude', 'Latitude', 'LA']).size().reset_index(name='frequency')
coord.plot(x="Longitude", y="Latitude", kind="scatter", c="frequency",
        colormap="YlOrRd")


#### By Local Authority

In [ ]:
la =Accidents.groupby('LA').size().reset_index(name="Accidents")

la = la.sort_values(['Accidents'], ascending=False)


la.head()

In [ ]:
#plot the distribution of accidents according to vehicle propulsion codes
la_20 = la[:20].plot(x="LA", y="Accidents", kind="barh",
        colormap="jet_r")

la_20.set_title('\nTop 20 local Authorities with the highest frequency of accidents\n', fontsize=14, fontweight='bold')
la_20.set(xlabel='frequency', ylabel='')
la_20.legend(bbox_to_anchor=(1.25, 0.98), frameon=False)

sns.despine(top=True, right=True, left=True, bottom=True);
figs.append(la_20)

import reverse_geocoder as rg

#### By geographic area

In [ ]:

'''Engineer Distance(location_area) Feature from the coordinates using a spatial distance algorithm. 
To facilitate further analysis'''

# we adopt cityblock(manhattan) algorithm because it is appropriate for roads.

location_area = [] # Create an empty list
for i in range(len(Accidents)): # iterate through dataframe rows
    dist = cityblock(Accidents['Longitude'][i],Accidents['Latitude'][i]) # Get the cityblock distance measures
    km = 6371*(math.pi/180)*dist # Convert the distance to appropriate SI unit(kilometre)
    
    location_area.append(km) # Append distances to empty list
Accidents['location_area'] = location_area # create  a location_area column

In [ ]:
#Engineered feature is scaled to get the difference of the accident location(in km) closest to the null island 
#(point where the longitude and latitude is equals to zero as reference) from other locations in Kilometre.

dist_from_null = Accidents['location_area'] - Accidents['location_area'].min() # Get the difference in area by deducting the minimum value from other values
Accidents['dist_from_null'] = dist_from_null # Use the kilometre difference list to create a feature in Analysis dataframe
Accidents['dist_from_null'].describe()

In [ ]:
Accidents['dist_from_null'].mode()

The locations with the highest accident occurence in the year 2019 is around 201, 617, 618 qnd 666 kms from our reference point respectively

In [ ]:
Accidents['dist_from_null'].value_counts().index.tolist()[:9]

In [ ]:
#Getting the dataframe of the locations(Kilometre) within the area of the top 4 locations with highest frequencies
risky_sites = Accidents[(Accidents['dist_from_null']== 201.920202) & 
                        (Accidents['dist_from_null']==617.699048) | (Accidents['dist_from_null']==618.363438) & 
                        (Accidents['dist_from_null']==666.766145)]

#### By geographic clusters

In [ ]:
def Elbow_method(data, n, init, n_init, max_iter, random_state):
    """
        Function to determine the optimal value of number of clusters(k) in KMeans clustering.
        It accepts a dataframe,
        range of values, initial_value, number of intialized, maximum iteration and random values
    """
    kmeans_kwargs ={
        'init': init,
        'n_init': n_init,
        'max_iter': max_iter,
        'random_state': random_state
    } # Initialise Kmeans algorithm
    sse = [] # empty list to store number of K
    for i in range(1,n): # loop through n 
        kmeans = KMeans(n_clusters=i, **kmeans_kwargs) # Initialize Kmeans
        kmeans.fit(data)# fit in the data
        sse.append(kmeans.inertia_) #append the number of inertia
    plt.style.use("fivethirtyeight") # Plotting style
    plt.plot(range(1,n),sse) # plot graph
    plt.xticks(range(1,n)) # x axis range
    plt.xlabel("Number of Clusters")  # Label x axis
    plt.ylabel("SSE") # Label y axis
    plt.show() # show plot

In [ ]:
# Use elbow method to determine the optimal number of clusters for Kmeans clustering
new_location = Accidents[['Longitude', 'Latitude']]
Elbow_method(new_location,15,'random', 10, 300, 42)

from math import radians
df['Lat'] = df['Latitude'].apply(radians)
df['Long'] = df['Longitude'].apply(radians)

clusterer = hdbscan.HDBSCAN(min_cluster_size=N, metric='haversine')
cluster_labels = clusterer.fit_predict(points)

In [ ]:
def Cluster(data, init, n_cluster, n_init, max_iter, random_state ) :
    """
        Function to perform KMeans clustering, a clustering algorithm in Machine Learning.
        Cluster algorithms use distance measures.
        Function accepts parameters;
        dataframe, initial value, number of clusters to form(k), number of initial values, maximum iterations,
        random state
    """
    print(data.isnull().sum()) # Confirm data has no missing values
    kmeans = KMeans(init = init, n_clusters = n_cluster, n_init = n_init, max_iter = max_iter, random_state = random_state) #Initialize kmean clustering
    kmeans.fit(data) # Fit model
    return kmeans.labels_ # Return label results

In [ ]:
#Use Kmeans clustering to engineer a new feature using its inertia_label as values
Accidents['loc_cluster'] = Cluster(new_location, 'random',4,10, 300, 42 )

from math import radians
from sklearn.cluster import DBSCAN

#coords = df.as_matrix(columns=['Latitude', 'Longitude'])
db = DBSCAN(eps=0.2, min_samples=5, algorithm='ball_tree', metric='haversine').fit_predict(np.radians(new_location))


plt.scatter(X['lat'], X['lng'], c=X['cluster'])
plt.show()

In [ ]:
Get_stats(Accidents, ['loc_cluster'])

Cluster 2 is the modal class with frequency of 56567 accidents. 

Cluster 2 constitutes 48.12% of all accidents.

In [ ]:
# convert the cluster labels to categorical features
Accidents['loc_cluster'] = Accidents['loc_cluster'].astype('category')

# plot the accidents distribiution by location clusters
plt.figure(figsize=(8,6))

clust=sns.scatterplot(data= Accidents, x='Longitude',y='Latitude', hue='loc_cluster') # Plot the graph
plt.title('Distribution of accidents in various location clusters')
plt.show()
figs.append(clust)

### Road Types

In [ ]:
############################### Investigating accidents on various road types ##########################################

# Convert Road_Type feature values to road type names
road = {1:'Roundabout',2:'One way street',3:'Dual carriage',6:'Single carriageway',7:'Slip road',
           9:'Unknown',12:'One way street/Slip road',
             -1:'missing'}
Accidents['Road_Type'] = Convert(Accidents,'Road_Type', road)

In [ ]:
road_tp= sns.countplot(data= Accidents, y='Road_Type', color = 'green')
plt.tick_params(axis='x', rotation=90)
plt.title('Accidents on various Road Types in the UK(2019)', size= 14)
plt.show()
figs.append(road_tp)

In [ ]:
Get_stats(Accidents, ['Road_Type'])

From the summary statistics and plot of the distribution we can deduce that a high number of accidents occurred on Single carriageways With the frequency of occurences at 85320 accidents accounting for 72.6% of all accidents. The increased risk on such roads might be explained by the lack on lane dividers on single carriageways. 

### Lighting conditions

In [ ]:
############################# Investigating accidents with various lighting conditions #####################################

# Convert light_Conditions values to light conditions names
light = {1:'Daylight',4:'Darkness-lights lit',5:'Darkness - light unlit',6:'Darkness - no lighting',
        7:'Darkness - lighting unknown',
             -1:'missing'}
Accidents['Light_Conditions'] = Convert(Accidents,'Light_Conditions', light)

In [ ]:
#Plot light conditions distribution
light_cond=sns.countplot(data= Accidents, x='Light_Conditions')
plt.tick_params(axis='x', rotation=45)
plt.title('Accidents during various lighting conditions')
plt.show()
figs.append(light_cond)

In [ ]:
Get_stats(Accidents, ['Light_Conditions'])

From the summary statistics and plot of the distribution we can deduce that a high number of accidents occur in normal daylight with the frequency of occurences at 83511 accidents accounting for 71.05% of all accidents. Traffic is busier during the day than at night with more people driving in daylight and directly increasing the risk of accidents.

### Road surface conditions

In [ ]:
############################### Investigating accidents with road surface conditions ######################################

#Convert Road_Surface_Conditions feature values to Road_Surface_Conditions names
surface = {1:'Dry',2:'Wet or damp',3:'Snow',4:'Frost or ice',5:'Flood over 3cm. deep',
           6:'Oil or diesel',7:'Mud',
             -1:'missing'}
Accidents['Road_Surface_Conditions'] = Convert(Accidents,'Road_Surface_Conditions', surface)

In [ ]:
#Plot for frequencies of road surface conditions
surf_cond=sns.countplot(data= Accidents, y='Road_Surface_Conditions', color = 'red')
plt.title('Accidents on various Road_Surface_Conditions', size= 14)
plt.tick_params(axis='x', rotation=90)
plt.show()
figs.append(surf_cond)

In [ ]:
Get_stats(Accidents, ['Road_Surface_Conditions'])

70% of all accidents happen on dry roads. Less people drive in bad weather including raining weather as previously discussed explaining this observation.

### Special situations

In [ ]:
############################### Investigating accidents in special situations ##########################################

#Convert values in special conditions on site to special conditions names
special = {0:'None',1:'Auto traffic signal-out',2:'Auto signal part defective',3:'Road sign or marking defective or obscured',4:'Roadworks',5:'Road surface defective',
           6:'Oil or diesel',7:'Mud',
             -1:'missing'}
Accidents['Special_Conditions_at_Site'] = Convert(Accidents,'Special_Conditions_at_Site', special)

In [ ]:
#Plotting Special_Condition_at_Site
site_cond=sns.countplot(data= Accidents, y='Special_Conditions_at_Site', color = 'green')
plt.title('Accidents by site Condition', size = 14)
#plt.tick_params(axis='x', rotation=45)
plt.show()
figs.append(site_cond)

In [ ]:
Get_stats(Accidents, ['Special_Conditions_at_Site'])

Less accidents(3.7%) happen at sites with special Conditions than they do or regular roads.Special conditions like defects on the roads are not a norm and as such do not contribute largely to the risk of accidents in comparison with regular roads.

# H. Driver description and Accidents

### How do driver related variables affect the outcome (e.g., age of the driver, and the purpose of the journey)?

We examine the relationship between specific factors and the nature of the accident(severity) using apriori knowledge

We also plot the relationships

In [ ]:
#map values to Casualty_Severity class
severity = {1:'Fatal',2:'Serious',3:'Slight'}
Analysis['Casualty_Severity'] = Convert(Analysis,'Casualty_Severity', severity)


### Age of Driver

In [ ]:
# Bin age group and generate column
Analysis['Age_group_of_Driver'] = pd.cut(x=df['Age_of_Driver']
                         , bins=[-1, 0, 5, 10, 15, 20, 25, 35, 45, 55, 65, 75, 101]
                         , labels=['unknown','0-5','6-10','11-15','16-20','21-25','26-35',
          '36-45', '46-55','56-65','66-75','75+'], include_lowest=True)
Analysis[['Age_group_of_Driver', 'Age_of_Driver']].head()

In [ ]:
#################################### investigating relationship of features #################################### 

def get_dummies(data:pd.DataFrame, feature:list, prefix:list,df_name:list, new_data:str):
    """
    The function is to join two dummied features together, 
    it accept as parameters; data, feature list, prefix list, df_name list
    and new data type.
    """
    df_name[0] = pd.get_dummies(data[feature[0]],prefix=prefix[0])
    df_name[1] = pd.get_dummies(data[feature[1]],prefix=prefix[1])
    new_data = df_name[0].join(df_name[1])
    return new_data

In [ ]:
# Define rule
Age_severity = get_dummies(Analysis,['Casualty_Severity','Age_group_of_Driver'],['Severity','Driver'],['sev','drv'],'sev_drv')

In [ ]:
def Apriori_association(data, min_support, min_threshold, metric):
    """
    This function deduces association rules by implementing apriori algorithm.
    Parameters are a dataframe, minimum support,
    minimum threshold and metrics as parameters
    """
    frequent_set = apriori(data, min_support= min_support, use_colnames=True)
    rules = association_rules(frequent_set, metric=metric,min_threshold=min_threshold)
    return rules

In [ ]:
# Test relationship
Apriori_association(Age_severity, 0.1, 0.5, 'lift')

The result is two associations with high support and confidence for age group of driver and severity of casualty with support of 0.1 and confidence of 0.9. However the relationship is for drivers whose age groups are unknown.

we confirm a relationship between the age group of drivers and the severity of casualties involved in the accidents.

In [ ]:
#group to examine impact

by_age = Analysis.groupby(['Age_group_of_Driver', 'Casualty_Severity']).size()

by_age = by_age.unstack('Casualty_Severity')

by_age

In [ ]:
#visualize relationship
fig, ax = plt.subplots(figsize=(8, 5))

by_age.plot(kind='barh', ax=ax, stacked=True, cmap='copper')
ax.set_title('\nAccident Severity by Age group of driver\n', fontsize=14, fontweight='bold')
ax.set(xlabel='frequency', ylabel='')
ax.legend(bbox_to_anchor=(1.25, 0.98), frameon=False)

sns.despine(top=True, right=True, left=True, bottom=True);

### Purpose of journey

In [ ]:
#map values to Journey_Purpose_of_Driver class

Purpose = {1:'Journey as part of work',2:'Commuting to/from work',3:'Taking pupil to/from school',4:'Pupil riding to/from school',5:'Other',
           6:'Not known',7:'Other/Not known (2005-10)',
             -1:'Data missing or out of range'}
Analysis['Journey_Purpose_of_Driver'] = Convert(Analysis,'Journey_Purpose_of_Driver',Purpose)

In [ ]:
# Define rule
Journey_casualty = get_dummies(Analysis, ['Journey_Purpose_of_Driver','Casualty_Severity'],['purpose','severity'],
                                    ['pur','sev'],'pur_sev')

In [ ]:
# Test relationship
Apriori_association(Journey_casualty,0.1,0.5,'lift')

In [ ]:
#group to examine impact

by_purpose = Analysis.groupby(['Journey_Purpose_of_Driver', 'Casualty_Severity']).size()

by_purpose = by_purpose.unstack('Casualty_Severity')

by_purpose

strong associations exist between journeys associated with work and slight casualty severity. However, the strongest association with support of 0.47 and confidence of 0.85 is between journeys with no known purpose and slight casualty severity. 

This implies that the purpose of journey had little bearing on the severity of accidents.

In [ ]:
#visualize relationship
fig, ax = plt.subplots(figsize=(6, 4))

by_purpose.plot(kind='barh', ax=ax, stacked=True, cmap='PuBu')
ax.set_title('\nAccident Severity by purpose of journey\n', fontsize=14, fontweight='bold')
ax.set(xlabel='frequency', ylabel='')
ax.legend(bbox_to_anchor=(1.25, 0.98), frameon=False)

sns.despine(top=True, right=True, left=True, bottom=True);

In [ ]:
#Define a function to save plots as png files
def save_plot(file_paths, figures): #accepts list of file paths and a list of figures
    figs_to_save = {} #create an empty dictionary
    for i,j in zip(file_paths, figures):
        figs_to_save[i]= j #assign file path as key and figure as value and populate figs_to_save
    for file_path, figure in figs_to_save.items(): #for items in figs_to_save
        try:
            figure.savefig(file_path) #save figure to file path
        except AttributeError: #process attribute error if figure is not a FacetGrid object
            figure.get_figure().savefig(file_path) #attempt saving again
            
    return 'The plots have been saved' #confirm completion

In [ ]:
plots = figs 
paths= []
for i in range(len(figs)):
    paths.append(f"Figures/{i}.png")
    
save_plot(paths, plots)

# I. Predicting Accidents

### Can we make predictions about when and where accidents will occur, and the severity of the injuries sustained from the data supplied to improve road safety? How well do our models compare to government models? 

We attempt to develop models that will accurately predict the following:

1. Time of Accidents

2. Location of accidents

3. Severity of Accidents.

Objectives:

i. create dataframes of relevant features and preprocess

ii. perform feature engineering to select relevant features

iii. build relevant models according to task (regression/classification)

iv. fit models

v. make predictions

vi. Evaluate models (**Metrics are RMSE, r2 score, Accuracy & precision score to take into consideration data imbalance**)

vii. Compare results with baselines

In [ ]:
######################################### Defining useful functions ###################################################

def encode_data(df, cols, module, encoder = None) :
    """
        Function to Encode Categorical Features, various methods for encoding are included in this function, to choose a
        method,
        Parameters :
        df = Dataframe
        cols = column or columns needed
        package = method of packages to use which can be : le, ce_oe,oh,du,fe,mt
        we provide the following short form of methods with its meaning as follows :
        le = Label Encoding
        ce_oe = categorical encoders Ordinal Encoding
        oh = One hot encoding
        du = dummy encoding
        fe = frequency encoding
        mt = mean target encoding

        encoder = external method needed for encoding- default set to none
    """
    encoded_df = df[cols] #Define dataframe to be encoded
    if module == "le" : # Check for the condition of label encoding
        encoded_df = encoded_df.apply(encoder.fit_transform) # Encode the dataframe
        return encoded_df # return the encoded feature
    
    elif module == "ce_oe" : # Check for categorical encoding condition
        encoded_df = encoder.fit_transform(encoded_df) # Fit and transform the data using categorical encoding
        return encoded_df # Return encoded feature
    
    elif module == "oh" : # Check for one hot encoding condition
        encoded_df = pd.DataFrame(encoder.fit_transform(encoded_df).toarray()) # fit and transform features using one hot encoding
        return encoded_df # Return the encoded data
    
    elif module == "du" : # Check condition for get dummy 
        encoded_df = pd.get_dummies(df[cols]) # Dummy the features
        return encoded_df # Return encoded features
    
    elif module == 'fe' : # Check for the frequency encoder
        encoded_df = df[cols] # Create a dataframe for the encoder
        y = encoded_df.groupby(cols).size().reset_index() # Groupby the frequency of the values of the features to encode
        y.columns = [cols[0], 'freq'+cols[0]] # Change the column names to the groupby names
        encoded_df = pd.merge(encoded_df,y,on = cols[0], how = 'left') # Merge with dataframe
        return encoded_df # return encoded features
    
    elif module == "mt" : #Check for mean target encoding
        encoded_df = df[cols] # Create its dataframe
        x = encoded_df.groupby([cols[0]])[cols[1]].sum().reset_index() # Get the feature sum by grouping by the feature and target then getting the sum.
        x = x.rename(columns={cols[1]:cols[0]+"_"+cols[1]+"_sum"})# rename the columns to the grouped columns names

        y = encoded_df.groupby([cols[0]])[cols[1]].count().reset_index()# get feature counts by groupby the column to encode and the target
        y = y.rename(columns={cols[1] :cols[0]+"_"+cols[1]+"_Count"}) # Rename the count columns

        z = pd.merge(x,y,on = cols[0], how = 'inner') # Merge the columns together

        z['Target_enc_levels'] = z[cols[0]+"_"+cols[1]+"_sum"]/z[cols[0]+"_"+cols[1]+"_Count"] # Create a new feature by geting the mean 
        encoded_df = pd.merge(encoded_df,z,on = cols[0], how = 'left') # Merge it with the dataframe

        return encoded_df # Return the encoded feature

In [ ]:
def Scaler(df, cols, method):
    """
        Function to scale a dataframe using various of methods.
        This function accepts a dataframe, 
        columns to scale and,
        the
        methods for scaling them, 

        It processes and returns a dataframe of scaled values
        The methods are :
        mmx == Minmax Scaler
        sts == Standard Scaler
        rbs == Robust Scaler
    """
    scaled_df = df[cols] # Create a dataframe to scale
     #for minmax scaler
    if method.lower() == "mmx": # Check for minmax scaling condition
        scaler = MinMaxScaler() # Define the scaler
        scaled_df = pd.DataFrame(scaler.fit_transform(scaled_df)) # Fit and transform data with the scaler
        scaled_df.columns = cols # Assign column names to the scaled columns
        return scaled_df # Return scaled data
    
    #for standard scaler
    elif method.lower() == "sts": # Check for standard scaler condition
        scaler = StandardScaler() # Define the scaler
        scaled_df = pd.DataFrame(scaler.fit_transform(scaled_df)) # Fit and transform the data using the scaler
        scaled_df.columns = cols # Assign column names to the scaled data
        return scaled_df# Return the dataframe
    
    #for robust scaler
    elif method.lower() == "rbs": # Check condition for robust scaler
        scaler = RobustScaler() # Define the scaler
        scaled_df = pd.DataFrame(scaler.fit_transform(scaled_df))# fit and transform the data using the scaler
        scaled_df.columns = cols # Assign column names to the dataframe
        return scaled_df # Return dataframe

In [ ]:
def PrincipalCA(data, n_components, columns):
    """
        Function to perform the Principal Component Analysis on the dataframe.
        A feature engineering technique that uses distance
        to get component from data, and takes care of overlapping features.
        Parameters are, a dataframe, amount of component 
        columns to be derived and the initial columns names
    """
    pca = PCA(n_components=n_components) # Initializing PCA
    principalComponents = pca.fit_transform(data) # Fit and transform data using the initialised PCA
    principalDf = pd.DataFrame(data= principalComponents,columns = columns) # Convert the transformed data to a dataset
    return principalDf # Return transformed dataframe

In [ ]:
def split_data(X, y, random_state, test_size):
    """
        This is a function to split the dataframe into train and test data.
        parameters are features, label, random_state, test_size, and variables to store split data.
    """
    Xtrain, Xtest, y_train, y_test = train_test_split(X,y,random_state=random_state,test_size=test_size) # Split data and assign to variables
    print(Xtrain.shape) # Print shape for train features
    print(Xtest.shape) # Print shape for test features
    print(y_train.shape)# Print shape for train labels
    print(y_test.shape) # Print shape for test labels
    return Xtrain, Xtest, y_train, y_test # Return variables for data

In [ ]:
def regressors(train_features, train_label, test_features, test_label, models):
    """
    Function builds multiple regression models using different algorithms with default parameters 
    and returns their metrics.
    it accepts train_features, train_label, test_features, test_label, and the list of models
    """
    scores=pd.DataFrame(columns=['Model', 'RMSE', 'r2_score'])
   # predictions=[]
    
    for model in models:
        model.fit(train_features,train_label) # Fit model
        y_pred = model.predict(test_features) # predict test labels using the test features
        model_name = type(model).__name__ #get model name
       
        
        #predictions.append(y_pred)
        
         # Evaluate model with RMSE and R2 score
        scores= scores.append({'Model':model_name,
                               'RMSE':np.sqrt(mean_squared_error(test_label,y_pred)),
                               'r2_score': r2_score(test_label,y_pred)}, ignore_index=True)
        
    #create dataframe of models and predictions    
    #pred_df = pd.DataFrame(predictions.transpose(), columns = ['LR', 'KN', 'RF', 'GB'])


    
    return scores#, pred_df

In [ ]:
def classifiers(train_features, train_label, test_features, test_label, models):
    """
    Function builds multiple classifier models using different algorithms with default parameters 
    and returns their metrics.
    it accepts train_features, train_label, test_features, test_label, and the list of models
    """
    scores=pd.DataFrame(columns=['Model', 'Accuracy', 'Precision'])
    #predictions= []
    
    for model in models:
        model.fit(train_features,train_label) # Fit model
        y_pred = model.predict(test_features) # predict test labels using the test features
        model_name = type(model).__name__ #get model name
       
        
        #predictions.append(y_pred)
        
         # Evaluate model with RMSE and R2 score
        scores= scores.append({'Model':model_name,
                               'Accuracy':accuracy_score(test_label,y_pred),
                               'Precision': precision_score(test_label,y_pred, average='weighted')}, ignore_index=True)
        
    #create dataframe of models and predictions    
    #pred_df = pd.DataFrame (predictions.transpose(), columns = ['LR', 'KN', 'RF', 'GB'])


    
    return scores#, pred_df

In [ ]:
def Stacking(task, X:list, Y:list, n_split):
    if task.lower() == 'regression':
        
        # get a stacking ensemble of base models
        level0 = list()
        level0.append(('lr', LinearRegression()))
        level0.append(('knn', KNeighborsRegressor()))
        level0.append(('rfc', RandomForestRegressor()))
        level0.append(('GBr', GradientBoostingRegressor()))
        
        # define meta learner model
        level1 = LinearRegression()        

        #simple kfold since target may be continuous
        kfold = KFold(n_splits=n_split, shuffle=True, random_state=42)
        
        # define the stacking ensemble
        stack_model = StackingRegressor(estimators=level0,
                                        final_estimator=level1, passthrough=False,
                                        cv=kfold)

        stack_model.fit(X[0], Y[0])
        R_pred = stack_model.predict(X[1])
        
        rmse =np.sqrt(mean_squared_error(Y[1], R_pred))
        r2 =r2_score(Y[1],R_pred)
        
        print(f' The RMSE of the stack model is {rmse}')
        print(f' The r2_score of the stack model is {r2}')
        return stack_model
    
    elif task.lower() == 'classification':

        # get a stacking ensemble of base models
        level0 = list()
        level0.append(('xgb', XGBClassifier()))
        level0.append(('knn', KNeighborsClassifier()))
        level0.append(('rfc', RandomForestClassifier()))
        level0.append(('gbc', GradientBoostingClassifier()))
        
        # define meta learner model
        level1 = XGBClassifier()        

        
        kfold = KFold(n_splits=n_split, shuffle=True, random_state=42)

        stack_model = StackingClassifier(estimators=level0,
                                        final_estimator=level1, passthrough=False,
                                        cv=kfold)

        stack_model.fit(X[0], Y[0])
        C_pred = stack_model.predict(X[1])
        
        acc= accuracy_score(Y[1], C_pred)
        prec=precision_score(Y[1], C_pred, average = 'weighted')

        print(f' The Accuracy of the stack model is {acc}')
        print(f' The Precision of the stack model is {prec}')
        return stack_model
    
    else:
        print(f'please define an appropriate task')

In [ ]:
 def Cross_val(X, y, n_splits, n_repeats, model_list, scoring):
    """
    Function to compare various models using cross validation
    """
    cv = RepeatedKFold(n_splits=n_splits, n_repeats= n_repeats, random_state=1)
    model_scores=[]
    
    for model in model_list:
        score = cross_val_score(model, X, y, scoring=scoring, cv=cv, n_jobs=-1, error_score='raise')
        score = np.mean(scores)
        model_scores.append(score)
    return model_scores
        
    

In [ ]:
# Updating original dataframe

#To reconcile Accidents and Analysis together
df = pd.merge(Accidents,Analysis, on = "Accident_Index", how = "left",  suffixes=('', '_drop')) 

# drop duplicated rows
df.drop_duplicates(keep='first',inplace=True)

# drop duplicated columns
df.drop([col for col in df.columns if 'drop' in col], axis=1, inplace=True)

#fill in missing values
fill_missing(df) #For forward fill
fill_missing(df, 'backfill') #For backward fill

df.info() #Check df especially for missing values
#no need to forcefully drop vehicle reference as it is not relevant for analysis and predictions

In [ ]:
#define model lists
Regressors = [LinearRegression(), KNeighborsRegressor(), RandomForestRegressor(), GradientBoostingRegressor()]

Classifiers = [XGBClassifier(), KNeighborsClassifier(), RandomForestClassifier(), GradientBoostingClassifier()]

### Predicting time of accidents
>Regression task

In [ ]:
# define dataset based on intuitive feature importance
Time = df[['Weather_Conditions','week','Hour','Day_of_Week','Light_Conditions','Special_Conditions_at_Site',
           'decimal_time', 'Location_Easting_OSGR','Location_Northing_OSGR','LA']]

In [ ]:
Time.info()

In [ ]:
#categorical variables
Time_cat = ['Weather_Conditions','Light_Conditions','Special_Conditions_at_Site','LA']

# encode categotical data
Time[Time_cat] = encode_data(Time, Time_cat, "le",LabelEncoder())
Time.head()

In [ ]:
time_X = Time.drop('decimal_time',axis=1)
time_y = Time['decimal_time']
time_X_scaled = Scaler(time_X, time_X.columns, 'rbs')
pca_time = PrincipalCA(time_X_scaled, 4, ['col1','col2','col3','col4'])
pca_time.head()

In [ ]:
# split the data
time_X_train,time_X_test,time_y_train,time_y_test = split_data(pca_time, time_y, 42, 0.25)

In [ ]:
# fit regression models
scores = regressors(time_X_train,time_y_train,time_X_test,time_y_test, Regressors )

In [ ]:
scores

In [ ]:
# stack models
time_stack = Stacking('regression',[time_X_train,time_X_test],[time_y_train,time_y_test],5)

In [ ]:
# append scores to model df
scores= scores.append({'Model':'stacked_model',
                        'RMSE':0.7115149,
                       'r2_score':0.9808938}, ignore_index=True)
scores

In [ ]:
time_stack

### Predicting location of accidents
>Classification task


In [ ]:
# define dataset based on intuitive feature importance
Location_data = df[['Location_Easting_OSGR','Location_Northing_OSGR','Road_Surface_Conditions','Road_Type',
                   'Urban_or_Rural_Area','loc_cluster', "dist_from_null", 'Day_of_Week', 'LA']]

In [ ]:
#Check dataset
Location_data.info()

In [ ]:
#categorical variables
Location_cat = ['Location_Easting_OSGR','Location_Northing_OSGR','Road_Surface_Conditions','Road_Type',
                   'Urban_or_Rural_Area', 'Day_of_Week', 'LA']

#Encode Categorical Features
Location_data[Location_cat] = encode_data(Location_data,Location_cat, "le",LabelEncoder())


In [ ]:
loc_X = Location_data.drop('loc_cluster',axis=1) #features
loc_y = Location_data['loc_cluster'] #label

#scale data to normalise
loc_Xscaled = Scaler(loc_X, loc_X.columns, 'rbs')

# Engineer Features using PCA
pca_loc = PrincipalCA(loc_Xscaled, 4, ['col1','col2','col3','col4'])
pca_loc.head()

In [ ]:
# Split data using train test split
loc_X_train,loc_X_test,loc_y_train,loc_y_test = split_data(pca_loc, loc_y, 42, 0.25)

In [ ]:
# Build and evaluate classification models
scores2= classifiers(loc_X_train,loc_y_train,loc_X_test,loc_y_test, Classifiers)

In [ ]:
scores2

In [ ]:
# stack models
loc_stack1 = Stacking('classification',[loc_X_train,loc_X_test],[loc_y_train,loc_y_test,],5)

In [ ]:
# append scores to model df
scores2= scores2.append({'Model':'stacked_model',
                               'Accuracy':0.999194,
                               'Precision':0.999194}, ignore_index=True)
scores2

>Regression task

Predicting distance from null using coordinates in Euclidean distance is ideal since the data is local to a small area(The UK) 

In [ ]:
# Engineer features using PCA
location_X = Location_data.drop('dist_from_null',axis=1)
location_y = Location_data ['dist_from_null']

# Scale data
location_data = Scaler(location_X, location_X.columns, 'rbs')
pca_loc2 = PrincipalCA(location_data, 4, ['col1','col2','col3', 'col4'])

In [ ]:
# Split data using train_test_split
loc2_X_train,loc2_X_test,loc2_y_train,loc2_y_test = split_data(pca_loc2, location_y, 42, 0.25)

In [ ]:
#fit regression models
scores3 = regressors(loc2_X_train,loc2_y_train,loc2_X_test,loc2_y_test, Regressors)

In [ ]:
scores3

In [ ]:
# stack models
loc_stack2=Stacking('regressors',[loc2_X_train,loc2_X_test],[loc2_y_train,loc2_y_test],5)

In [ ]:
# append scores to model df
scores3= scores3.append({'Model':'stacked_model',
                        'RMSE':4.178288,
                         'r2':0.999734, ignore_index=True})
scores3

### Predicting severity of accidents
>Clasification task

In [ ]:
# define dataset based on intuitive feature importance

Severity = df[['Speed_limit','Weather_Conditions','Age_of_Casualty','Age_of_Driver','Casualty_Class'
              ,'Propulsion_Code','Casualty_Severity','Pedestrian_Movement','Light_Conditions',
               'Pedestrian_Location','Day_of_Week','Hour','LA','Carriageway_Hazards']]

In [ ]:
#categorical variables
Severity_cat = ['Weather_Conditions','Propulsion_Code','Casualty_Class','Propulsion_Code',
                'Carriageway_Hazards','Light_Conditions',
                'Casualty_Severity','Pedestrian_Movement','Pedestrian_Location','LA']

#Encode Categorical Features
Severity[Severity_cat] = encode_data(Severity,Severity_cat, "le",LabelEncoder())

In [ ]:
sev_X = Severity.drop('Casualty_Severity',axis=1) # Features
sev_y = Severity['Casualty_Severity'] # Label

#scale data to normalise
sev_X_scaled = Scaler(sev_X, sev_X.columns, 'rbs')
# Engineer Features using PCA
pca_sev = PrincipalCA(sev_X_scaled, 4, ['col1','col2','col3','col4'])

In [ ]:
# Split data using train test split
sev_X_train,sev_X_test,sev_y_train,sev_y_test = split_data(pca_sev, sev_y, 42, 0.25 )

In [ ]:
# Build and evaluate classification models
scores4 = classifiers(sev_X_train,sev_y_train,sev_X_test,sev_y_test, Classifiers)

In [ ]:
scores4

In [ ]:
# stack models
sev_stack=Stacking('classification',[sev_X_train,sev_X_test],[sev_y_train,sev_y_test,],5)

In [ ]:
# append scores to model df
scores4= scores4.append({'Model':'stacked model',
                               'Accuracy': 0.849396,
                               'Precision':0.804295}, ignore_index=True)
scores4

### Baseline comparison- Government Predictions

>Government models were used to predict for only the slight or serious severity class, Manipulation of the data is required in order to compare the predictions with our model's.  In addition, while, we are only extrapolating for 2019, the governments predictions run for several years with millions of values. We will also only deal with relevant entries from the government adjustment file.

In [ ]:
# accident severity data
cas = df[['Accident_Index','Speed_limit','Weather_Conditions','Age_of_Casualty','Age_of_Driver','Casualty_Class',
              'Propulsion_Code','Casualty_Severity','Pedestrian_Movement',
               'Pedestrian_Location','Day_of_Week','Hour','LA','Carriageway_Hazards']]

# get df of govt. adjustment file
cas_adj = pd.read_csv('cas_adjustment_lookup_2019.csv')


In [ ]:
print(len(cas), len(cas_adj))

''' length of both dataframes do not match to make a comparison of values. The difference is too large to merge both 
further analysis will therefore be carried out on just casualties and accidents data eliminating all 
vehicle features'''

In [ ]:
#To merge Accidents and casualty together
cas_acc = pd.merge(Accidents,casualties, on = "Accident_Index", how = "left",  suffixes=('', '_drop')) 

# drop duplicated rows
cas_acc.drop_duplicates(keep='first',inplace=True)

# drop duplicated columns
cas_acc.drop([col for col in df.columns if 'drop' in col], axis=1, inplace=True)

#fill in missing values
fill_missing(cas_acc) #For forward fill
fill_missing(cas_acc, 'backfill') #For backward fill

cas_acc.head(2)

In [ ]:
cas = cas_acc[['Accident_Index','Speed_limit','Weather_Conditions','Age_of_Casualty','Casualty_Class',
              'Casualty_Severity','Pedestrian_Movement','Light_Conditions','Casualty_Type',
               'Pedestrian_Location','Day_of_Week','Hour','LA','Carriageway_Hazards']]

# drop the fatal class
cas = cas[(cas['Casualty_Severity'] == 2) | (cas['Casualty_Severity'] == 3)]

# match accident index column for both files
cas_adj.rename(columns={'accident_index': 'Accident_Index'}, inplace=True)

# get index for 2019 entries
sev_indx = cas_acc['Accident_Index'].unique()

#get 2019 from adjustments
cas_adj = cas_adj[cas_adj['Accident_Index'].isin(sev_indx)]

cas_adj['Accident_Index'].unique()

In [ ]:
# merge both dfs to match lengths(important for evaluation)
cas_sev_adj = pd.merge(cas, cas_adj)

cas_sev_adj.head()

We get evaluate the adjusted predictions against the actual values to get the baseline scores for the predictions

In [ ]:
# convert adj predictions to binary classes and generate column
cas_sev_adj['sev'] = [ 1 if (i>=0.5 and j<0.5) else 0 for i,j in zip(cas_sev_adj['Adjusted_Serious'], cas_sev_adj['Adjusted_Slight'])]
cas_sev_adj['sev'].unique()

In [ ]:
cas_sev_adj['Casualty_Severity'].unique()

In [ ]:
# get the actual classes to match the now binary adjusted predictions, serious accidents are the positive class 1
cas_sev_adj['Cas_Sev'] = [ 1 if x== 2 else 0 for x in cas_sev_adj['Casualty_Severity']]
cas_sev_adj['Cas_Sev'].unique()

In [ ]:
acc=accuracy_score(cas_sev_adj['Cas_Sev'], cas_sev_adj['sev'])
prec=precision_score(cas_sev_adj['Cas_Sev'], cas_sev_adj['sev'])


In [ ]:
# evaluate government models using accuracy score and precision
print(f'government model predicted accidents severity for 2019 with an accuracy of {acc}')
print(f'government model predicted accidents severity for 2019 with an precision of {prec}')

In [ ]:
'''Predicting with the best existing model for the scenario'''
# data
Sev_data = cas_sev_adj[['Speed_limit','Weather_Conditions','Age_of_Casualty','Casualty_Class',
              'Casualty_Severity','Pedestrian_Movement','Light_Conditions','Casualty_Type',
               'Pedestrian_Location','Day_of_Week','Hour','LA','Carriageway_Hazards','Cas_Sev']]


In [ ]:
#categorical variables
Sev_data_cat = ['Speed_limit','Weather_Conditions','Casualty_Class',
                'Casualty_Type','Pedestrian_Movement','Light_Conditions',
               'Pedestrian_Location','Speed_limit','Cas_Sev','LA']

#Encode Categorical Features
Sev_data[Sev_data_cat] = encode_data(Sev_data,Sev_data_cat, "le",LabelEncoder())

In [ ]:
sev_data_X = Sev_data.drop('Cas_Sev',axis=1) # Features
sev_data_y = Sev_data['Cas_Sev'] # Label

#scale data to normalise
sev_data_X_scaled = Scaler(sev_data_X, sev_data_X.columns, 'rbs')

# Engineer Features using PCA, set n_components to 4 to capture significant variance
pca_sev_data = PrincipalCA(sev_data_X_scaled, 4, ['col1','col2','col3','col4'])

#split data without further feature engineering
sev_data_X_train,sev_data_X_test,sev_data_y_train,sev_data_y_test = split_data(pca_sev_data, sev_data_y, 42, 0.2 )

In [ ]:
# define model (using current best model)
model = sev_stack

In [ ]:
# fit  model
model.fit(sev_data_X_train,sev_data_y_train)

In [ ]:
# predict
stack_pred = model.predict(sev_data_X_test)

# evaluate
model_acc = accuracy_score(sev_data_y_test, stack_pred)
model_prec = precision_score(sev_data_y_test, stack_pred)

print(f'The stacking model predicted accidents severity for 2019 with an accuracy of {model_acc}')
print(f'The stacking model predicted accidents severity for 2019 with a precision of {model_prec}')
